# TASK-1 

This task is to provide a JSON output which summarised the number of fires per administrative boundary per year in Telangana. The json output should include the following keys - adm_name, year, fireCount. Please use the same key names, and the boundary names

First, all necessary libraries has to be installed like pandas, rtree, pygeos and pandas.<br>
pandas: This library can be used for converting a set of data into a dataframe and do necessary operations on them.
geopandas: This library can be used as pandas for spatial data and for their vizualization <br>
pygeos, rtree: These are the dependent libraries which are has to be installed for geopandas<br>

In [ ]:
!pip install pandas
!pip install rtree
!pip install pygeos
!pip install geopandas

## Importing Libraries

geopandas and pandas are imported as gpd and pd respectively to use their functions with shortforms

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
#---------------------THIS PART IS USED TO WORK FROM DRIVE---------------#
#from google.colab import drive
#drive.mount('/content/drive/')

#import os
#os.chdir(r'/content/drive/MyDrive/undp')

## Creating DataFrame with given datasets


Firstly, we need to download the administrative boundaries of India from [GADM level 3](https://gadm.org/), and further features are extracted for the Telangana state boundary using QGIS. <br>
<br>
As fire_points are read as comma-separated values, we have to assign latitude and longitude values to use as a geodataframe. We will create a pandas data frame _'fire_points'_ and extract the longitude and latitude column from it. Further, we will assign geometry to each point using its longitude, latitude values in new geodataframe _'newdf'_.To create a geodataframe (_'pointstodist'_) which can show each fire point with administrative boundaries in which it is present, the administrative boundaries shapefile (dist.shp) and fire points in Telangana boundary are read using geopandas (_'adm_name'_ and _'nwedf'_)and joined together using _'sjoin'_ function of geopandas with the intersection of keys from both GeoDataFrames.

In [ ]:
#shapefile of districts of Telangana
adm_name = gpd.read_file('dist.shp')
#shapefile of fire data given
fire_points = pd.read_csv('telangana_fires.csv') 

#--------------Geodataframe for fire points--------------#
newdf = gpd.GeoDataFrame(fire_points, geometry=gpd.points_from_xy(fire_points.longitude, fire_points.latitude), crs=4326)

#--------------- using inner join to merge to geoDataFrames-------------#
pointstodist = gpd.sjoin(newdf, adm_name, how="inner", op='intersects')

## Creating JSON File:

The JSON output should include the following keys - adm_name, year, fireCount. For this, we need to rename geodataframe column _'NAME_3'_ as _'adm_name'_. Next, we will fetch years for each fire point from _'acq_date'_ and dump it into a new column named _'year'_. Now we have to count the fire points for each year and administrative boundary. This is done by creating an empty column in the data frame named as _'FireCount'_ and summing fire points grouping by _'adm_name'_ & _'year'_ .
Now, we will create a data frame that will be containing only three columns of the previous data frame named as _'adm_name'_, _'year'_ and _'fireCount'_ to dump in a JSON file named as _'output1.json'_.<br>
<br>

In [ ]:
#renaming column by districts
pointstodist = pointstodist.rename(columns={"NAME_3":"adm_name"})
#getting year from the date
pointstodist['year'] = pd. DatetimeIndex(pointstodist['acq_date']).year
#declaring firecount column by 1
pointstodist['fireCount']=1
#suming the occurence of fire by grouping district and year
new_df = pointstodist.groupby(['adm_name','year']).sum()  
new_df = new_df.reset_index()

#creating new Dataframe to dumping json
json_pd = new_df[['adm_name','year','fireCount']]
#dumping to json file
json_pd = json_pd.to_json('output1.json', orient="records", lines=True)     